In [ ]:
! pip install datasets transformers accelerate evaluate degender-pronoun

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install scikit-learn==1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.0 which is incompatible.


In [ ]:
import sys, gc
import scipy
import numpy as np
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))

scipy.stats.itemfreq=monkeypath_itemfreq

import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from degender_pronoun import degenderizer
from datasets import Dataset
import tqdm


import pandas as pd

degender_pronouns = {
    ' mr ': ' mx ',
    ' mrs ': ' mx ',
    ' ms ': ' mx ',
    ' miss ': ' mx ',
    ' mister ': ' mx ',
}

degender_nouns = {
    ' man ': ' person ',
    ' men ': ' persons ',
    ' woman ': ' person ',
    ' women ': ' persons ',
    " man's ": " person's",
    " men's ": " person's",
    " woman's ": " person's",
    " women's ": " person's",
    " gentleman ": " person ",
    " lady ": " person ",
    " gentleman's ": " person's ",
    " lady's ": " person's ",
}

def preprocess(df, data_column, preprocess_type):
    if preprocess_type == 'none':
        return df

    D = degenderizer()
    df[data_column] = df[data_column].apply(lambda x: D.degender(x) if len(x) > 5 else x)

    for k, v in degender_pronouns.items():
        df[data_column] = df[data_column].str.lower().replace(k,v)

    if preprocess_type == 'all':
        for k, v in degender_nouns.items():
            df[data_column] = df[data_column].str.lower().replace(k,v)
    return df

model_checkpoint = "ben-yu/roberta-base-finetuned-nlp-letters-s1_s2-pronouns-class-weighted"
batch_size = 16
task = "nlp-letters"
metric_name = "accuracy"
data_column = "TEXT"
model_name = model_checkpoint.split("/")[-1]
num_labels = 2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, low_cpu_mem_usage=True)
class_names = ['female', 'male']
def model_adapter(text):
    preds = []

    # use bert encoder to tokenize text
    encoded_input = tokenizer(text,
      return_tensors='pt',
      padding=True,
      truncation=True,
      max_length=model.config.max_position_embeddings-2)
    # run the model
    output = model(**encoded_input)
    # by default this model gives raw logits rather
    # than a nice smooth softmax so we apply it ourselves here
    output = np.argmax(output[0].softmax(1).detach().numpy()[:][0])
    return class_names[output]


data_column = 's1_s2'
preprocessing_type = 'all'
dataset_path = "sentence_sets.csv"
df = pd.read_csv(dataset_path, encoding='unicode_escape')
df = preprocess(df, data_column, preprocessing_type)
dataset = Dataset.from_pandas(df).rename_column("APPLICANT_GENDER", "label").class_encode_column("label").train_test_split(test_size=0.2)

class_preds = []
gt_preds = []
text_preds = []
output_frame = []
for data in tqdm.tqdm(dataset['test']):
  text = data[data_column]
  gc.collect()
  class_pred = model_adapter(text)
  gt_pred = class_names[data["label"]]
  output_frame.append((text, class_pred, gt_pred))
new_df = pd.DataFrame(output_frame, columns=["Text", "Prediction", "GT"])
new_df.to_csv(f"results_{data_column}_{model_name}.csv")

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Casting to class labels:   0%|          | 0/4411 [00:00<?, ? examples/s]

100%|██████████| 883/883 [18:54<00:00,  1.29s/it]
